In [1]:
import pandas as pd
import numpy as np

In [8]:
#Connecting to data
meter = pd.read_csv('Demand 1 meter.csv')
weather = pd.read_csv('demand_1.2_weather.csv')

<H1>Data Preparation</H1>

In [9]:
#Weather data preparation
data_weather = weather.set_index(pd.DatetimeIndex(weather['date_time']))
data_weather = data_weather[['temperature', 'dewpoint', 'cloud_cover', 'humidity', 'wind_speed', 'wind_direction', 'precipitation']]
data_weather.head()

,temperature,dewpoint,cloud_cover,humidity,wind_speed,wind_direction,precipitation
date_time,,,,,,,
2018-01-01 00:00:00,-2.0,-12.0,0,62,11.5,290,0.0
2018-01-01 01:00:00,-2.0,-13.0,0,59,16.1,310,0.0
2018-01-01 02:00:00,-3.0,-14.0,0,59,10.4,310,0.0
2018-01-01 03:00:00,-4.0,-14.0,0,62,11.5,300,0.0
2018-01-01 04:00:00,-4.0,-15.0,0,59,13.8,310,0.0


In [10]:
#meter data preparation
data_meter = meter[['Date', 'Hour','Volume']]
data_meter['Date'] = pd.to_datetime(data_meter['Date'])
data_meter['Hour'] = pd.to_datetime(data_meter['Hour'], format='%H')
data_meter['Hour'] = data_meter['Hour'].dt.time
data_meter['date_time'] = pd.to_datetime(data_meter.Date.astype(str) + ' ' + data_meter.Hour.astype(str))
data_2 = data_meter[['date_time' ,'Volume']]
data_2 = data_2.set_index(pd.DatetimeIndex(data_2['date_time']), drop=True)
data_2 = data_2['Volume']
data_2.head()

C:\Users\BIKRAM~1\AppData\Local\Temp/ipykernel_14300/1750588931.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_meter['Date'] = pd.to_datetime(data_meter['Date'])


ValueError: unconverted data remains: 4

<H1>Data Upsampling</H1>

<H2>Weather Data Upsampling</H2>

<H3>To convert to minutes than convert to sencond. eg. 5min = 300s</H3>

In [11]:
data_weather_2 = data_weather[data_weather.index <= '2021-12-31 23:00:00']
#data_weather_2 = data_weather[data_weather.index >= '2018-01-01 00:00:00']

In [15]:
data_weather_2 = data_weather_2.resample('1800S').asfreq()
data_2_weather = data_weather_2.interpolate(method='linear')
data_2_weather.head()

,temperature,dewpoint,cloud_cover,humidity,wind_speed,wind_direction,precipitation
date_time,,,,,,,
2018-01-01 00:00:00,-2.0,-12.0,0.0,62.0,11.50,290.0,0.0
2018-01-01 00:30:00,-2.0,-12.5,0.0,60.5,13.80,300.0,0.0
2018-01-01 01:00:00,-2.0,-13.0,0.0,59.0,16.10,310.0,0.0
2018-01-01 01:30:00,-2.5,-13.5,0.0,59.0,13.25,310.0,0.0
2018-01-01 02:00:00,-3.0,-14.0,0.0,59.0,10.40,310.0,0.0


In [35]:
data_2_weather.tail()

,temperature,dewpoint,cloud_cover,humidity,wind_speed,wind_direction,precipitation
date_time,,,,,,,
2021-12-31 22:40:00,40.697558,38.532910,98.657174,91.893194,6.347489,219.017355,0.000008
2021-12-31 22:45:00,40.706118,38.580719,98.700965,92.124126,6.186604,219.648188,0.000007
2021-12-31 22:50:00,40.725405,38.640681,98.771251,92.384904,5.993065,220.352209,0.000006
2021-12-31 22:55:00,40.756379,38.713531,98.870206,92.676528,5.764867,221.134465,0.000003
2021-12-31 23:00:00,40.800000,38.800000,99.000000,93.000000,5.500000,222.000000,0.000000


In [36]:
data_2_weather.to_csv('Demand 1 5 Min Weather.csv')

In [32]:
data_2_weather.shape

(105109, 7)

<H2>Meter data Upsampling</H2>

In [13]:
data_2 = data_2[data_2.index <= '2021-12-31 23:00:00']
#data_2 = data_2[data_2.index >= '2019-01-01 00:00:00']

In [14]:
data_3 = data_2.resample('300S').asfreq()
data_3 = data_3.interpolate(method='cubic')
data_3 = data_3.to_frame()
data_3.head()

,Volume
date_time,
2018-01-01 00:00:00,8313.000000
2018-01-01 00:05:00,8267.852757
2018-01-01 00:10:00,8223.997548
2018-01-01 00:15:00,8181.422976
2018-01-01 00:20:00,8140.117645


In [15]:
data_3['Date'] = data_3.index.date
data_3['Hour'] = data_3.index.hour
data_3['Minute'] = data_3.index.minute
data_3 = data_3.set_index(pd.DatetimeIndex(data_3['Date']), drop = True)
data_3.tail()

,Volume,Date,Hour,Minute
Date,,,,
2021-12-31,7006.138712,2021-12-31,22,40
2021-12-31,6977.933828,2021-12-31,22,45
2021-12-31,6950.171754,2021-12-31,22,50
2021-12-31,6922.858481,2021-12-31,22,55
2021-12-31,6896.000000,2021-12-31,23,0


In [16]:
data_3.to_csv("Demand 1 5 Min Meter.csv")

<H1>Weather Data Downsampling</H1>

<H2>To downsample data to day use 'D', month='M', Year='Y'</H2>

<H3>Weather Data Downsampling</H3>

In [16]:
data_weather_3 = data_weather.resample('D').mean()
data_weather_3.head()

,temperature,dewpoint,cloud_cover,humidity,wind_speed,wind_direction,precipitation
date_time,,,,,,,
2018-01-01,0.108333,-13.825000,2.291667,52.208333,15.245833,291.125000,0.000000
2018-01-02,6.779167,-5.450000,10.833333,57.500000,12.462500,259.625000,0.000000
2018-01-03,14.654167,0.554167,48.666667,55.000000,8.712500,227.625000,0.000000
2018-01-04,16.783333,9.300000,99.666667,76.625000,21.970833,188.708333,0.021667
2018-01-05,5.795833,-6.237500,47.708333,57.791667,20.854167,274.625000,0.000417


In [6]:
data_weather_3.to_csv("Demand 1 Daily Weather.csv")

<H3>Meter Data Downsampling</H3>

In [7]:
data_meter_down = data_2.resample('D').sum()
data_meter_down.to_frame()
data_meter_down.head()

date_time
2018-01-01    203384.0
2018-01-02    214048.0
2018-01-03    207680.0
2018-01-04    206323.0
2018-01-05    203598.0
Freq: D, Name: Volume, dtype: float64

In [8]:
data_meter_down.to_csv('Demand 1 Daily Meter.csv')